# Training Collaborative Experts on MSR-VTT
This notebooks shows how to download some code that trains a modification of a Collaborative Experts model with BERT + CLS + NetVLAD on the MSR-VTT Dataset.


## Setup

*   Download Code and Dependencies
*   Import Modules
*   Download Language Model Weights
*   Download Datasets
*   Generate Encodings for Dataset Captions 



### Code Downloading and Dependency Downloading
*   Specify tensorflow version
*   Clone repository from Github
*   `cd` into the correct directory
*   Install the requirements




In [1]:
%tensorflow_version 2.x

In [2]:
!git clone https://github.com/googleinterns/via-content-understanding.git

Cloning into 'via-content-understanding'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 7211 (delta 8), reused 14 (delta 5), pack-reused 7189
Receiving objects: 100% (7211/7211), 850.07 KiB | 2.25 MiB/s, done.
Resolving deltas: 100% (4750/4750), done.


In [3]:
%cd via-content-understanding/videoretrieval/

/content/via-content-understanding/videoretrieval


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 778kB 13.3MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 3.0MB 24.0MB/s 
     |████████████████████████████████| 1.1MB 51.2MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a233886c70ff4eef6f0d9103fa5c21fe2ec5c5ff367f5aef22fecdc47dd4cef8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Found existing installation: reque

In [5]:
!pip install --upgrade tensorflow_addons

     |████████████████████████████████| 1.1MB 8.8MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


### Importing Modules

In [6]:
import tensorflow as tf
import languagemodels
import train.encoder_datasets
import train.language_model
import experts
import datasets
import datasets.msrvtt.constants
import os
import models.components
import models.encoder
import helper.precomputed_features
from tensorflow_addons.activations import mish  
import tensorflow_addons as tfa
import metrics.loss

### Language Model Downloading

*   Download BERT



In [7]:
bert_model = languagemodels.BERTModel()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


### Dataset downloading


*   Downlaod Datasets
*   Download Precomputed Features



In [8]:
datasets.msrvtt_dataset.download_dataset()

Note: The system `curl` is more memory efficent than the download function in our codebase, so here `curl` is used rather than the download function in our codebase.

In [9]:
url = datasets.msrvtt.constants.features_tar_url
path = datasets.msrvtt.constants.features_tar_path
os.system(f"curl {url} > {path}") 

0

In [10]:
helper.precomputed_features.cache_features(
    datasets.msrvtt_dataset,
    datasets.msrvtt.constants.expert_to_features,
    datasets.msrvtt.constants.features_tar_path,)

speech: (32, 300) | (29, 300)
ocr: (49, 300) | (5, 300)
densenet: (2208,) | (1, 2208)
audio: (29, 128) | (1, 128)


### Encoding Generation

* Generate Encodings for MSR-VTT

In [11]:
train.language_model.generate_and_cache_encodings(
    bert_model, datasets.msrvtt_dataset)

## Training


*  Build Train Datasets
*  Initialize Models
*  Compile Encoders
*  Fit Model
* Test Model


### Datasets Generation

In [12]:
experts_used = [
  experts.i3d,
  experts.r2p1d,
  experts.resnext,
  experts.senet,
  experts.speech_expert,
  experts.ocr_expert,
  experts.audio_expert,
  experts.densenet,
  experts.face_expert]

In [13]:
train_ds, valid_ds, test_ds = (
    train.encoder_datasets.generate_language_model_fine_tuning_datasets(
        bert_model, datasets.msrvtt_dataset, experts_used))

### Model Initialization

In [14]:
class MishLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return mish(inputs)

In [ ]:
mish(tf.Variable([1.0]))

In [16]:
text_encoder = models.components.TextEncoder(
    len(experts_used),
    num_netvlad_clusters=28,
    ghost_clusters=1,
    language_model_dimensionality=768,
    encoded_expert_dimensionality=512,
    residual_cls_token=True,
)

In [17]:
video_encoder = models.components.VideoEncoder(
    num_experts=len(experts_used),
    experts_use_netvlad=[False, False, False, False, True, True, True, False, False],
    experts_netvlad_shape=[None, None, None, None, 19, 43, 8, None, None],
    expert_aggregated_size=512,
    encoded_expert_dimensionality=512,
    g_mlp_layers=3,
    h_mlp_layers=0,
    make_activation_layer=MishLayer)

In [18]:
encoder = models.encoder.EncoderForLanguageModelTuning(
    video_encoder,
    text_encoder,
    0.05,
    [1, 5, 10, 50],
    20,
    bert_model.model,
    64)

### Encoder Compliation

In [19]:
def build_optimizer(lr=0.001):
    learning_rate_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps=1000,
        decay_rate=0.95,
        staircase=True)

    return tf.keras.optimizers.Adam(learning_rate_scheduler)

In [20]:
encoder.compile(build_optimizer(5e-5), metrics.loss.bidirectional_max_margin_ranking_loss)

In [21]:
train_ds_prepared = (train_ds
  .shuffle(7000)
  .batch(32, drop_remainder=True)
  .prefetch(tf.data.experimental.AUTOTUNE))
valid_ds_prepared = (valid_ds
  .prefetch(tf.data.experimental.AUTOTUNE)
  .batch(497 * 20, drop_remainder=True)
  .cache()

In [22]:
encoder.language_model.trainable = True
encoder.video_encoder.trainable = True
encoder.text_encoder.trainable = True

### Model fitting

In [ ]:
encoder.fit(
    train_ds_prepared,
    #validation_data=valid_ds_prepared,
    epochs=250,
)

### Tests

In [51]:
captions_per_video = 20
num_videos_upper_bound = 100000 

In [ ]:
ranks = []

for caption_index in range(captions_per_video):
    batch = next(iter(test_ds.shard(captions_per_video, caption_index).batch(
        num_videos_upper_bound)))
    video_embeddings, text_embeddings, mixture_weights = encoder.forward_pass(
        batch, training=False)
    
    similarity_matrix = metrics.loss.build_similarity_matrix(
        video_embeddings,
        text_embeddings,
        mixture_weights,
        batch[-1])
    rankings = metrics.rankings.compute_ranks(similarity_matrix)
    ranks += list(rankings.numpy())

In [ ]:
def recall_at_k(ranks, k):
    return len(list(filter(lambda i: i <= k, ranks))) / len(ranks)

In [ ]:
median_rank = sorted(ranks)[len(ranks)//2]

In [ ]:
mean_rank = sum(ranks)/len(ranks)

In [ ]:
print(f"Median Rank: {median_rank}")

In [ ]:
print(f"Mean Rank: {mean_rank}")

In [ ]:
for k in [1, 5, 10, 50]:
    recall = recall_at_k(ranks, k)
    print(f"R@{k}: {recall}")